In [1]:
import numpy as np
import openmm as mm
import mdtraj as md
from openmm import unit
import openmm.app as app
import os
from openmmtools import states, mcmc
import openmmtools
import os
import os.path as osp
from ReplicaExchangeProtocol import ReplicaExchange


if osp.exists('output-1.nc'): os.system('rm output-1.nc')

In [2]:
forcefield=['amber99sb.xml','tip3p.xml']
forcefield=['amber96.xml','tip3p.xml']
constraints = app.HBonds
nonbonded_cutoff = 0.9*unit.nanometer
switch_distance = 0.75*unit.nanometer
nonbonded_method = app.PME
implicit_solvent = False
solvated = True
hydrogen_mass = None

# Define initial constants of the input pdb file
friction = 0.1 / unit.picoseconds
pressure = 1.0 * unit.atmosphere
temperature = 300*unit.kelvin
timestep = 2.0 * unit.femtosecond

# Load pdb file
pdb = app.PDBFile('chignolin.pdb')
traj = md.load('chignolin.pdb')

# Set periodic boundary condition
xdist = traj.xyz[:,:,0].max() - traj.xyz[:,:,0].min()
ydist = traj.xyz[:,:,1].max() - traj.xyz[:,:,1].min()
zdist = traj.xyz[:,:,2].max() - traj.xyz[:,:,2].min()

dimensions = np.zeros((3,3))
dimensions[0][0] = xdist
dimensions[1][1] = ydist
dimensions[2][2] = zdist

pdb.topology.setPeriodicBoxVectors(dimensions)

# Define system
create_system_kwargs = dict(
    removeCMMotion=True,
    nonbondedMethod=nonbonded_method,
    nonbondedCutoff=nonbonded_cutoff,
    switchDistance=switch_distance,
    constraints=constraints,
    hydrogenMass=hydrogen_mass,
    rigidWater=True,
)


ff = app.ForceField(*forcefield)
system = ff.createSystem(pdb.topology, **create_system_kwargs)

#sampler_state = mcmc.SamplerState(positions=system.positions)
#thermodynamic_state = mcmc.ThermodynamicState(system=system, temperature=298*unit.kelvin)


barostat = mm.MonteCarloBarostat(pressure, temperature)
force_id = system.addForce(barostat)

In [10]:
platform = mm.Platform.getPlatformByName("CUDA")
platform_properties = {"DeviceIndex": "0", "Precision": "mixed"}

n_replicas = 2

protocol = {'temperature': [300,302,304,306] * unit.kelvin}
thermodynamic_states = states.create_thermodynamic_state_protocol(system,protocol)



sampler_states = list()
for i_t,_ in enumerate(thermodynamic_states):
    #pos = unit.Quantity(start_pos,unit=unit.angstrom)
    pos = unit.Quantity(traj.xyz, unit=unit.nano)
    sampler_states.append(openmmtools.states.SamplerState(positions=pos))

langevin_move = mcmc.LangevinSplittingDynamicsMove(
    timestep = timestep,
    collision_rate = friction,
    n_steps = 50,
    reassign_velocities=False,
    n_restart_attempts=20,
    splitting="V R O R V"
)

# Define class for replica exchange
parallel_tempering = ReplicaExchange(
    thermodynamic_states=thermodynamic_states,
    sampler_states=sampler_states,
    mcmc_move=langevin_move
)

In [7]:
positions, forces, acceptance = parallel_tempering.run(5000,save=True)

0


AttributeError: 'SiPrefix' object has no attribute 'iter_base_dimensions'

In [16]:
langevin_move.apply(thermodynamic_states[0], sampler_states[0])

AttributeError: 'SiPrefix' object has no attribute 'iter_base_dimensions'

In [17]:
system.express_unit()

AttributeError: 'System' object has no attribute 'express_unit'